In [84]:
import pandas as pd

data = pd.read_csv("seven_visits_train_new.csv")
data6 = pd.read_csv("data6.csv")

C:\Users\tminh\AppData\Local\Temp\ipykernel_17708\2759761197.py:4: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  data6 = pd.read_csv("data6.csv")


In [85]:
import pandas as pd

# Perform the merge as before
merged_df = pd.merge(data, data6, on=['NACCID', 'VISITDATE'], suffixes=('', '_6'))

# Update columns that exist in both DataFrames
common_columns = data.columns.intersection(data6.columns).drop(['NACCID', 'VISITDATE'])

for col in common_columns:
    data[col] = merged_df[col + '_6'].combine_first(data[col])

# Clean up the extra columns from data6 that were added during the merge
data = data[data.columns.drop(list(data.filter(regex='_6')))]


In [86]:
giveaway_feats = ['MEMORY', 'ORIENT', 'JUDGMENT', 'COMMUN', 'HOMEHOBB', 'PERSCARE', 'CDRGLOB', 'NACCMMSE']
time_feats = ["VISITDATE", "COURSE", "VISITDAY", "VISITMO", "VISITYR"]
irrelevant_feats = ['visitorder',
 'NACCUDSD',
 'NACCDAYS',
 'NACCMMSE',
 'NACCAVST',
 'NACCFDYS',
 '...7',
 'NACCADC',
 'interval',
 'NACCVNUM',
 'patient_id',]

drop_feats = giveaway_feats + time_feats + irrelevant_feats

In [87]:
data = data.drop(columns = drop_feats)
data.to_csv('seven_visits_train_imputed_with_data6_temp.csv')

In [77]:
data_filled = data.fillna(0)

In [80]:
categorical_cols = data_filled.select_dtypes(exclude=['number']).columns

In [83]:
data_filled.to_csv("fully_filled_seven_visits.csv")

In [51]:
categorical_cols

Index(['NACCID', 'NACCAMS', 'PSYCDISX'], dtype='object')

In [75]:
cols_with_na = data.isna().any()
na_columns = data.columns[cols_with_na]
cleaned = data.loc[:, ~cols_with_na]

In [ ]:
cleaned.columns

In [69]:
six = pd.read_csv('data6.csv')

C:\Users\tminh\AppData\Local\Temp\ipykernel_19488\4119197334.py:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  six = pd.read_csv('data6.csv')


In [70]:
[_ for _ in six.columns if _ not in data.columns ]

['DECSUB...133',
 'DECSUB...174',
 'visit_number',
 'cluster',
 'var_cdrsum.x',
 'classification.x',
 'progression.x',
 'var_cdrsum.y',
 'classification.y',
 'progression.y',
 'var_cdrsum',
 'classification',
 'progression']

In [72]:
data.columns

Index(['NACCID', 'VISITDATE', 'patient_id', 'NACCVNUM', 'NACCAVST', 'interval',
       '...7', 'SEX', 'HISPANIC', 'HISPOR',
       ...
       'NACCUDSD', 'NACCDAYS', 'NACCFDYS', 'COURSE', 'VISITDAY', 'VISITMO',
       'VISITYR', 'visitorder', 'NACCADC', 'IMPAIRMENTS'],
      dtype='object', length=221)

In [93]:
import pandas as pd

# Example DataFrame setup
data = {
    'NACCID': [1, 1, 1, 2, 2, 2],
    'constant_col_1': ['A', 'A', 'A', 'B', 'B', 'B'],
    'constant_col_2': ['X', 'X', 'X', 'Y', 'Y', 'Y'],
    'feature_col_A': [10, 20, 30, 40, 50, 60],
    'feature_col_B': [100, 200, 300, 400, 500, 600]
}


# List of constant columns
CONSTANT_COLUMNS = ['NACCID', 'constant_col_1', 'constant_col_2']

data = pd.DataFrame(data)
data.loc[:, 'row_number'] = data.groupby('NACCID').cumcount() + 1
flattened_df = data.pivot_table(index='NACCID', 
                                    columns='row_number', 
                                    values=data.columns.difference(CONSTANT_COLUMNS + ['row_number']))
flattened_df.columns = [f'{col}_{num}' for col, num in flattened_df.columns]
flattened_df.reset_index()


,NACCID,feature_col_A_1,feature_col_A_2,feature_col_A_3,feature_col_B_1,feature_col_B_2,feature_col_B_3
0,1,10,20,30,100,200,300
1,2,40,50,60,400,500,600


In [95]:
pd.read_csv('seven_visits_train_imputed_with_data6_temp.csv')['NACCID']

C:\Users\tminh\AppData\Local\Temp\ipykernel_17708\1552849762.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('seven_visits_train_imputed_with_data6_temp.csv')['NACCID']


0        NACC000034
1        NACC000034
2        NACC000034
3        NACC000034
4        NACC000034
            ...    
23725    NACC999854
23726    NACC999854
23727    NACC999854
23728    NACC999854
23729    NACC999854
Name: NACCID, Length: 23730, dtype: object

In [100]:
import pandas as pd

# Sample data
data = {
    'NACCID': [1, 1, 1, 2, 2, 2],
    'constant_col_1': ['A', 'A', 'A', 'B', 'B', 'B'],
    'constant_col_2': ['X', 'X', 'X', 'Y', 'Y', 'Y'],
    'feature_col_A': [10, 20, 30, 40, 50, 60],
    'feature_col_B': [100, 200, 300, 400, 500, 600]
}
self_X = pd.DataFrame(data)

CONSTANT_COLUMNS = ['constant_col_1', 'constant_col_2']

result = self_X.groupby('NACCID').apply(lambda x: pd.Series({
    **{col: x[col].iloc[0] for col in CONSTANT_COLUMNS},
    **{f"{col}_{i+1}": x[col].iloc[i] for col in ['feature_col_A', 'feature_col_B'] for i in range(len(x))}
})).reset_index()

print(result)


   NACCID constant_col_1 constant_col_2  feature_col_A_1  feature_col_A_2  \
0       1              A              X               10               20   
1       2              B              Y               40               50   

   feature_col_A_3  feature_col_B_1  feature_col_B_2  feature_col_B_3  
0               30              100              200              300  
1               60              400              500              600  


In [104]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Sample DataFrame and target column
df = pd.DataFrame({
    'feature1': [1, 2, 1, 4, 5, 6],
    'feature2': [3, 4, 2, 5, 6, 7],
    'progression': [0, 1, 0, 0, 1, 0]
})

# Split features and target
X = df.drop(columns=['progression'])
y = df['progression']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create and fit the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(random_state=42))
])

pipeline.fit(X_train, y_train)

# Make predictions and evaluate
y_pred = pipeline.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))


ImportError: cannot import name '_get_column_indices' from 'sklearn.utils' (C:\Users\tminh\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\__init__.py)

In [3]:
import numpy as np


# Generate a synthetic dataset
X, y = make_classification(n_classes=2, class_sep=2,
                           weights=[0.1, 0.9], n_informative=3, 
                           n_redundant=1, flip_y=0, n_features=20, 
                           n_clusters_per_class=1, n_samples=1000, 
                           random_state=42)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize SMOTE and RandomForestClassifier
smote = SMOTE(random_state=42)
classifier = RandomForestClassifier(random_state=42)

# Create a pipeline with SMOTE and the classifier
pipeline = Pipeline([
    ('smote', smote),
    ('classifier', classifier)
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        27
           1       1.00      1.00      1.00       273

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300



In [2]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [6]:
import pandas as pd
df = pd.read_csv('flattened_3.csv')
df_filtered = df[df['progression'] == 0]
df_filtered

,Unnamed: 0,SEX,HISPANIC,HISPOR,RACE,PRIMLANG,EDUC,NACCFAM,NACCMOM,NACCDAD,...,DIGBACLS_1,DIGBACLS_2,DIGBACLS_3,CDRSUM_1,CDRSUM_2,CDRSUM_3,IMPAIRMENTS_1,IMPAIRMENTS_2,IMPAIRMENTS_3,progression
0,0,2.0,0.0,8.0,1.0,1.0,15.0,1.0,1.0,1.0,...,4.0,4.0,4.0,0.5,1.5,1.5,0.0,0.0,0.0,0
1,1,1.0,1.0,4.0,6.0,2.0,4.0,1.0,0.0,0.0,...,5.0,5.0,5.0,11.0,11.0,0.0,1.0,1.0,0.0,0
2,2,2.0,0.0,8.0,5.0,1.0,20.0,1.0,0.0,0.0,...,6.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,4,1.0,0.0,8.0,1.0,1.0,14.0,0.0,0.0,0.0,...,2.0,3.0,5.0,3.5,4.0,0.0,0.0,1.0,0.0,0
5,5,2.0,0.0,8.0,1.0,1.0,18.0,1.0,1.0,0.0,...,5.0,5.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3382,3382,1.0,0.0,8.0,1.0,1.0,18.0,1.0,1.0,0.0,...,NaN,NaN,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3383,3383,2.0,0.0,8.0,1.0,1.0,13.0,1.0,1.0,0.0,...,NaN,NaN,NaN,0.5,0.0,0.5,0.0,0.0,0.0,0
3387,3387,2.0,0.0,8.0,1.0,1.0,14.0,1.0,1.0,1.0,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0
3388,3388,2.0,0.0,8.0,1.0,1.0,18.0,1.0,1.0,1.0,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0
